In [4]:
import os
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

task = 'GlobalLocal'
LAB_root = None

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

for sub in subjects:
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

NameError: name 'sub' is not defined

In [18]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

Loaded significant channels for subject D0057


In [22]:
import os
import json

# Define the subjects list
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0071', 
            'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# Initialize a dictionary to hold the counts
counts = {
    'total_filt_channels': 0,
    'total_good_channels': 0,
    'total_sig_channels': 0,
}

# Define the base path for your files
base_path = 'C:/Users/jz421/Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/'

# Loop through each subject to gather data
for sub in subjects:
    # Load the 'good' channels
    good_channels_path = os.path.join(base_path, 'freqFilt', 'figs', sub, f'channels_{sub}_GlobalLocal.txt')
    with open(good_channels_path, 'r') as file:
        good_channels = file.readlines()  # Adjust this line if the data is stored in a different format (e.g., JSON)
    num_good_channels = len(good_channels)
    
        
    # Populate the dictionary using get_sig_chans for each subject
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')
    num_sig_channels = len(sig_chans_per_subject[sub])
    
    filt_channels_path = os.path.join(base_path, 'clean', f'sub-{sub}', 'ieeg', f'sub-{sub}_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv')
    with open(filt_channels_path, 'r') as file:
        filt_channels = file.readlines()  # Adjust this line if the data is stored in a different format (e.g., JSON)
    num_filt_channels = len(filt_channels) - 1

    # Update the counts
    counts['total_filt_channels'] += num_filt_channels
    counts['total_good_channels'] += num_good_channels
    counts['total_sig_channels'] += num_sig_channels
    
    # Print the counts for the current subject
    print(f"Subject {sub}: Filt Channels = {num_filt_channels}, Good Channels = {num_good_channels}, Sig Channels = {num_sig_channels}")

# Print total counts across all subjects
print("Total across all subjects:", counts)


Loaded significant channels for subject D0057
Subject D0057: Filt Channels = 179, Good Channels = 176, Sig Channels = 87
Loaded significant channels for subject D0059
Subject D0059: Filt Channels = 185, Good Channels = 174, Sig Channels = 88
Loaded significant channels for subject D0063
Subject D0063: Filt Channels = 241, Good Channels = 235, Sig Channels = 43
Loaded significant channels for subject D0065
Subject D0065: Filt Channels = 221, Good Channels = 216, Sig Channels = 86
Loaded significant channels for subject D0071
Subject D0071: Filt Channels = 163, Good Channels = 157, Sig Channels = 114
Loaded significant channels for subject D0077
Subject D0077: Filt Channels = 135, Good Channels = 131, Sig Channels = 40
Loaded significant channels for subject D0090
Subject D0090: Filt Channels = 165, Good Channels = 157, Sig Channels = 41
Loaded significant channels for subject D0094
Subject D0094: Filt Channels = 241, Good Channels = 230, Sig Channels = 55
Loaded significant channels for